In [1]:
!nvidia-smi

Tue Jun 28 11:54:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.60.02    Driver Version: 510.60.02    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 6000     On   | 00000000:1A:00.0 Off |                  Off |
| 34%   26C    P8    32W / 260W |   2394MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Quadro RTX 6000     On   | 00000000:1C:00.0 Off |                  Off |
| 33%   

In [5]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import skimage.transform as st
import os
import gc
import warnings
from sklearn.metrics import classification_report, average_precision_score
from Parse_TFrecords import *
from define_model import *
from load_label import *
from utilities import *
 
print(tf.__version__)

warnings.filterwarnings("ignore")

2.4.0


# Transfer learning

In [4]:
with tf.device('/GPU:6'):
    BATCH_SIZE = 32

    record_file_train = ['copd_train_new.tfrecords', 'copd_emory_train.tfrecords']
    train_dataset = (tf.data.TFRecordDataset(
        record_file_train, buffer_size=BATCH_SIZE, compression_type=None, num_parallel_reads=32)
    .map(parse_TFrecord_train)
    .shuffle(total)
    .batch(BATCH_SIZE))

    record_file_val = ['copd_val_new.tfrecords', 'copd_emory_val.tfrecords']
    val_dataset = (tf.data.TFRecordDataset(
        record_file_val, buffer_size=BATCH_SIZE, compression_type=None, num_parallel_reads=32)
    .map(parse_TFrecord_train)
    .batch(BATCH_SIZE))

    record_file_test = ['copd_test_new.tfrecords', 'copd_emory_test.tfrecords']
    test_dataset = (tf.data.TFRecordDataset(
        record_file_test, buffer_size=BATCH_SIZE, compression_type=None, num_parallel_reads=32)
    .map(parse_TFrecord_test)
    .batch(BATCH_SIZE))

    y_test = get_data_label('test')

In [10]:
    

filename = ['copd_test_new.tfrecords']
# filename = 'copd_emory_test.tfrecords'

y = []

#load the test files
raw_dataset = tf.data.TFRecordDataset(filename)
for raw_record in raw_dataset:
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())

    print(example)
    break

features {
  feature {
    key: "Atelectasis"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "COPD"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "Cardiomegaly"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "Consolidation"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "Edema"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "Enlarged Cardiomediastinum"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "Fracture"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "Lung Lesion"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "Lung Opacity"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "No Finding"
    value {
      int64_list {
        valu

## Different architectures

In [5]:
with tf.device('/GPU:6'):
archis = ['DenseNet121', 'ResNet50V2', 'MobileNetV2', 'InceptionResNetV2', 'Xception', 'EfficientNetV2S',  'InceptionV3']

    for archi in archis:
        checkpoint_filepath = 'checkpoints_new/checkpoint_BCE_{i}'.format(i=archi)
        monitor_ = 'val_loss'

        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_filepath,
            save_best_only=True,
            save_weights_only=True,
            monitor=monitor_,
            mode='min')

        callback = [tf.keras.callbacks.LearningRateScheduler(scheduler),
                    tf.keras.callbacks.EarlyStopping(mode='min', patience=3, monitor=monitor_),
                    model_checkpoint_callback]

        model = load_model_from_pretrain(archi)

        loss_func = tf.keras.losses.BinaryCrossentropy(from_logits=True)

        model.compile(loss=loss_func,
                         optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics='AUC')

        model.fit(train_dataset, epochs=3, shuffle=True, validation_data=val_dataset, callbacks=callback, class_weight=class_weight)

        del model
        gc.collect()

Epoch 1/20


KeyboardInterrupt: 

In [ ]:
with tf.device('GPU:6'):
    archis = ['DenseNet121', 'ResNet50V2', 'MobileNetV2', 'InceptionResNetV2', 'Xception', 'EfficientNetV2S',  'InceptionV3']

    for archi in archis:
        print(archi)
        checkpoint_filepath = 'checkpoints_new/checkpoint_BCE_{i}'.format(i=archi)
        model = define_model(archi)
        model.load_weights(checkpoint_filepath)

        y_preds = model.predict(test_dataset)

        test_CI(y_preds, y_test)

        print(average_precision_score(y_test, y_preds, average=None))

        thresh = get_thresh(y_test, y_preds, 'Youden')
        print('thresh:', thresh)

        print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))

        del model, y_preds
        gc.collect()